# API Data Producer

For getting our API data and producing it for our consumers we send specific requests to our API based on the player ID and the season paramater.
We made use of the KafkaProducer object and simply send the raw data (with minimal changes) that we receive per request to the Kafka Broker:

## Import Packages

In [ ]:
import requests
import json
import time
from kafka import KafkaProducer

## Define Topic and instantiate Kafka Producer

In [ ]:
bootstrap_servers = 'kafka:9092'
topic_name = 'NBA-API-TOPIC'
producer = KafkaProducer(bootstrap_servers=bootstrap_servers, value_serializer=lambda x: json.dumps(x).encode('utf-8'))

We pre-define the URL of the API and the seasons we want to look at so that we do not send unnecessary requests:

In [ ]:
base_url = 'https://www.balldontlie.io/api/v1'
seasons = [1995, 1996, 1997, 2005, 2006, 2007, 2011, 2012, 2013]

## Functions to get Player and Season Data

In [ ]:
def get_players():
    for player_id in range(3093):
        for season in seasons:
            player_details = {
                'id': player_id,
                'first_name': None,  # Placeholder for first name
                'last_name': None  # Placeholder for last name
            }
            
            retries = 0
            max_retries = 5
            
            while True:
                player_url = f'{base_url}/players/{player_id}'
                response = requests.get(player_url)
                
                if response.status_code == 200:
                    player_data = response.json()
                    if 'first_name' in player_data:
                        player_details['first_name'] = player_data['first_name']
                    if 'last_name' in player_data:
                        player_details['last_name'] = player_data['last_name']
                    break  # Break the while loop if the response code is 200
                else:
                    retries += 1
                    if retries > max_retries:
                        break  # Break the while loop if maximum retries reached
            
            yield player_details, season

            
def get_season_stats(player_id, season):
    url = f'{base_url}/season_averages?season={season}&player_ids[]={player_id}'
    response = requests.get(url)
    
    if response.status_code == 200:
        try:
            season_stats = response.json()
            if 'data' in season_stats and season_stats['data']:
                return season_stats['data']
        except json.JSONDecodeError:
            pass
    
    return None

In this code block below we iterate over each player and then the seasons to get all possible data for the players and seasons we are interested in and then we simply send that data (in JSON format) to our Kafka Broker. We commented out the actual producing for now:

In [ ]:
for player_details, season in get_players():
    player_id = player_details['id']
    player_season_stats = get_season_stats(player_id, season)

    if player_season_stats is not None:
        full_name = f"{player_details['first_name']} {player_details['last_name']}"

        player_data = {
            'full_name': full_name,
            **player_season_stats[0]
        }
        print(player_data)
        #producer.send(topic_name, value=player_data)
        
        print(f"Sent: {player_id} - {full_name} - {season}")

#producer.flush()
#producer.close()